<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#准备工作" data-toc-modified-id="准备工作"><span class="toc-item-num">1&nbsp;&nbsp;</span>准备工作</a></span></li><li><span><a href="#开始训练验证数据" data-toc-modified-id="Setting-up-training-/-validation-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>开始训练验证数据</a></span></li></ul></div>

# 准备工作

确保已安装了所有适当的软件包。
请参阅有关安装说明的DeepBlast Readme。
另外，如果您使用的是GPU，请确保正确配置了CUDA和CUDNN。
在用于运行此笔记本的slurm环境中，加载了以下模块。

module load gcc cudnn cuda

大约预计15min

# 开始训练验证数据

以下用在TM-Align产生的一小部分结构对齐子集中运行DeepBlast为例子。

In [ ]:
import os
import pandas as pd
from deepblast.sim import hmm_alignments
import argparse

fname = '../data/tm_align_output_10k.tab'
cols = [
    'chain1_name', 'chain2_name', 'tmscore1', 'tmscore2', 'rmsd',
    'chain1', 'chain2', 'alignment'
]
align_df = pd.read_table(fname, header=None, sep='\s+')
align_df.columns = cols
n_alignments = align_df.shape[0]

模拟对齐将分为训练 /测试和验证。

In [ ]:
parts = n_alignments // 10
train_df = align_df.iloc[:parts * 8]
test_df = align_df.iloc[parts * 8:parts * 9]
valid_df = align_df.iloc[parts * 9:]

# save the files to disk.
if not os.path.exists('data'):
    os.mkdir('data')
    
train_df.to_csv('data/train.txt', sep='\t', index=None, header=None)
test_df.to_csv('data/test.txt', sep='\t', index=None, header=None)
valid_df.to_csv('data/valid.txt', sep='\t', index=None, header=None)

准备环境，以确保存在适当的输出目录以存储结果。

In [ ]:
from deepblast.trainer import DeepBLAST
from pytorch_lightning import Trainer

output_dir = 'struct_results'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

首先设置参数：

In [ ]:
from transformers import T5EncoderModel, T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

现在创建 DeepBLAST 模型  

In [ ]:
model = DeepBLAST(
    train_pairs=f'{os.getcwd()}/data/train.txt',  # training data
    test_pairs=f'{os.getcwd()}/data/test.txt',    # test data
    valid_pairs=f'{os.getcwd()}/data/valid.txt',  # validation data
    output_directory=output_dir,                  # output directory storing model + diagnostics
    batch_size=10,                                # number of alignments per training batch
    num_workers=30,                               # number of cores for manipulating training data
    layers=2,                                     # number of CNN layers for blosum matrix estimation
    learning_rate=5e-5,                           # learning rate
    loss='cross_entropy',                         # type of loss function 
    lm=model,                                     # pretrained language model 
    tokenizer=tokenizer                           # tokenizer for residues
)

最后训练模型

In [ ]:
trainer = Trainer(
    max_epochs=5,
    gpus=1,
    check_val_every_n_epoch=1,
    limit_train_batches=5,
    limit_val_batches=1
)

trainer.fit(model)

模型可以在Tensorboard中实现可视化， 图像可以显示损失函数值，准确性和对齐结果。

In [ ]:
!ls lightning_logs

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir lightning_logs